In [6]:
# はじめに
#
# この一連の欠損値の穴埋めをみてください

import numpy as np
import pandas as pd

# 更新用df
tbl = pd.DataFrame(
    [
    {"cd":"F001","val":None}, 
    {"cd":"E001","val":None},
    {"cd":"X001","val":None},
    {"cd":"F001","val":''}, 
    {"cd":"E001","val":''},
    {"cd":"X001","val":''},
    {"cd":"F001","val":100}, 
    {"cd":"E001","val":200},
    {"cd":"X001","val":300}
    ],columns=["cd","val"]
)
# 参照df その１
fmst = pd.DataFrame(
    [
    {"cd":"F001","val":1}, 
    {"cd":"F002","val":2},
    {"cd":"F003","val":3}
    ],columns=["cd","val"]
)
# 参照df その２
emst = pd.DataFrame(
    [
    {"cd":"E001","val":4}, 
    {"cd":"E001","val":5},
    {"cd":"E001","val":6}
    ],columns=["cd","val"]
)

emst

,cd,val
0,E001,4
1,E001,5
2,E001,6


In [7]:
# 期待する結果の更新用df状態です
tbl = pd.DataFrame(
    [
    {"cd":"F001","val":1}, 
    {"cd":"E001","val":4},
    {"cd":"X001","val":None},
    {"cd":"F001","val":1}, 
    {"cd":"E001","val":4},
    {"cd":"X001","val":None},
    {"cd":"F001","val":100}, 
    {"cd":"E001","val":200},
    {"cd":"X001","val":300}
    ],columns=["cd","val"]
)
tbl

,cd,val
0,F001,1.0
1,E001,4.0
2,X001,NaN
3,F001,1.0
4,E001,4.0
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0


In [13]:
# 'いけね、'(空文字)は欠損値ではないのか？この対処はこちらでやります。
tbl = pd.DataFrame(
    [
    {"cd":"F001","val":None}, 
    {"cd":"E001","val":None},
    {"cd":"X001","val":None},
    {"cd":"F001","val":''}, 
    {"cd":"E001","val":''},
    {"cd":"X001","val":''},
    {"cd":"F001","val":100}, 
    {"cd":"E001","val":200},
    {"cd":"X001","val":300}
    ],columns=["cd","val"]
)
tbl = tbl.replace({'': None})
tbl

,cd,val
0,F001,NaN
1,E001,NaN
2,X001,NaN
3,F001,NaN
4,E001,NaN
5,X001,NaN
6,F001,100.0
7,E001,200.0
8,X001,300.0
